In [35]:
# Prepare the Environment
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import math
import tiktoken
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Setup Hyperparameters
# Hyperparameters
batch_size = 4  # How many batches per training step
context_length = 16  # Length of the token chunk each batch
d_model = 64  # The vector size of the token embeddings
num_layers = 8  # Number of transformer blocks
num_heads = 4  # Number of heads in Multi-head attention # 我们的代码中通过 d_model / num_heads = 来获取 head_size
learning_rate = 1e-3  # 0.001
dropout = 0.1 # Dropout rate
max_iters = 5000  # Total of training iterations
eval_interval = 50  # How often to evaluate the model
eval_iters = 20  # How many iterations to average the loss over when evaluating the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Instead of using the cpu, we'll use the GPU if it's available.

TORCH_SEED = 1337
torch.manual_seed(TORCH_SEED)

In [3]:
# Prepare the Dataset
# download a sample txt file from https://huggingface.co/datasets/goendalf666/sales-textbook_for_convincing_and_selling/raw/main/sales_textbook.txt
if not os.path.exists('sales_textbook.txt'):
    url = 'https://huggingface.co/datasets/goendalf666/sales-textbook_for_convincing_and_selling/raw/main/sales_textbook.txt'
    with open('sales_textbook.txt', 'w') as f:
        f.write(requests.get(url).text)

with open('sales_textbook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# Step 1: Tokenization
# Using TikToken to tokenize the source text
encoding = tiktoken.get_encoding("cl100k_base")
tokenized_text = torch.tensor(encoding.encode(text)) # size of tokenized source text is 77,919
vocab_size = len(set(tokenized_text)) # size of vocabulary is 3,771
max_token_value = max(tokenized_text)

print(f"Tokenized text size: {len(tokenized_text)}")
print(f"Vocabulary size: {vocab_size}")
print(f"The maximum value in the tokenized text is: {max_token_value}")

Tokenized text size: 77919
Vocabulary size: 77919
The maximum value in the tokenized text is: 100069


In [7]:
# Step 2: Word Embedding
# Split train and validation
split_idx = int(len(tokenized_text) * 0.8)
train_data = tokenized_text[:split_idx]
val_data = tokenized_text[split_idx:]

# Prepare data for training batch
# Prepare data for training batch
data = train_data
idxs = torch.randint(low=0, high=len(data) - context_length, size=(batch_size,))
x_batch = torch.stack([data[idx:idx + context_length] for idx in idxs])
y_batch = torch.stack([data[idx + 1:idx + context_length + 1] for idx in idxs])
print(x_batch.shape, x_batch.shape)

torch.Size([4, 16]) torch.Size([4, 16])


In [67]:
# Step 3: Positional Encoding
# Define Token Embedding look-up table
token_embedding_lookup_table = nn.Embedding(max_token_value, d_model)

# Get X and Y embedding
x = token_embedding_lookup_table(x_batch.data)
y = token_embedding_lookup_table(y_batch.data)

# Define Position Encoding look-up table
position_encoding_lookup_table = torch.zeros(context_length, d_model) # initial with zeros with shape (context_length, d_model)
position = torch.arange(0, context_length, dtype=torch.float).unsqueeze(1)
# apply the sine & cosine
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
position_encoding_lookup_table[:, 0::2] = torch.sin(position * div_term)
position_encoding_lookup_table[:, 1::2] = torch.cos(position * div_term)
position_encoding_lookup_table = position_encoding_lookup_table.unsqueeze(0).expand(batch_size, -1, -1) #add batch to the first dimension

print("Position Encoding Look-up Table: ", position_encoding_lookup_table.shape)

# Add positional encoding into the input embedding vector
input_embedding_x = x + position_encoding_lookup_table # [4, 16, 64] [batch_size, context_length, d_model]
input_embedding_y = y + position_encoding_lookup_table

X = input_embedding_x

x_plot = input_embedding_x[0].detach().cpu().numpy()
print("Final Input Embedding of x: \n")
pd.DataFrame(x_plot)

Position Encoding Look-up Table:  torch.Size([4, 16, 64])
Final Input Embedding of x: 



,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.283404,0.338045,-1.433035,2.767594,0.576844,1.222869,1.151606,1.042083,1.343139,-0.339763,...,0.486147,1.545294,0.225854,-0.347028,-0.645690,2.254384,0.796207,1.492581,2.124295,1.222536
1,2.405423,1.134329,-0.297423,1.513847,-0.094999,1.010294,0.945422,-0.156378,0.798628,-0.492561,...,0.451860,0.310261,-1.565642,-1.741235,1.525025,-1.283047,0.545836,2.580286,1.007957,2.450777
2,0.587248,-1.585031,0.868417,0.717200,0.750380,0.331701,0.446977,1.132623,0.627753,0.099642,...,-2.053629,0.528360,0.559716,-0.131694,0.353959,0.769088,-1.257144,2.869510,0.556096,0.200955
3,1.825541,0.508325,3.189547,-3.090334,-0.790009,-0.605251,3.264065,-0.547575,1.291236,2.863118,...,-1.765736,1.504601,0.736996,3.704933,-0.113013,0.826981,0.393594,1.762055,0.384672,3.776245
4,-1.716024,-1.586166,0.198546,-0.353335,3.309299,0.301478,0.728683,1.458770,2.398213,0.092539,...,0.739426,2.699533,-0.676643,0.467648,0.635566,1.664962,-0.759751,1.407433,-0.478800,1.408918
5,-1.650472,0.311326,-0.700890,-0.460482,1.144396,0.241577,-0.143885,0.376465,0.179648,-0.371582,...,1.243632,-0.405581,1.085325,1.973006,0.290641,0.963295,-1.359209,2.193898,0.814181,1.235836
6,-0.176708,1.625126,-3.843431,0.226879,-0.763457,-0.964958,1.278523,-0.875336,-0.665511,-0.101888,...,0.091525,0.851619,0.267266,1.929255,-0.984592,1.801168,-0.403364,0.300297,-1.973638,0.862498
7,0.729852,2.036257,-1.206615,2.121351,-1.688091,0.052425,-0.156384,-1.596992,1.558975,0.324975,...,0.383543,0.338034,-0.095333,-0.785435,1.885006,0.938034,-0.616108,-0.222979,1.323855,0.622632
8,2.357656,0.037400,0.325845,1.146377,-0.941756,0.425242,0.890695,-1.161934,0.509555,0.397377,...,0.773701,2.106979,0.112369,1.145097,-0.666539,1.018525,0.091615,1.301296,-0.844504,0.503519
9,1.341450,-0.485298,1.107402,1.558929,-0.446929,-1.098254,-0.614124,0.109389,3.285658,-2.086398,...,1.565137,0.503009,-0.716071,1.805186,-0.334426,0.404759,0.504305,2.839343,0.346494,3.016829


In [11]:
# Prepare Q,K,V
# Prepare Query, Key, Value for Multi-head Attention

query = key = value = X # [4, 16, 64] [batch_size, context_length, d_model]

# Define Query, Key, Value weight matrices
Wq = nn.Linear(d_model, d_model)
Wk = nn.Linear(d_model, d_model)
Wv = nn.Linear(d_model, d_model)

Q = Wq(query) #[4, 16, 64]
Q = Q.view(batch_size, -1, num_heads, d_model // num_heads)  #[4, 16, 4, 16]

K = Wk(key) #[4, 16, 64]
K = K.view(batch_size, -1, num_heads, d_model // num_heads)  #[4, 16, 4, 16]

V = Wv(value) #[4, 16, 64]
V = V.view(batch_size, -1, num_heads, d_model // num_heads)  #[4, 16, 4, 16]

In [12]:
# Transpose q,k,v from [batch_size, context_length, num_heads, head_size] to [batch_size, num_heads, context_length, head_size]
# The reason is that treat each batch with "num_heads" as its first dimension.
Q = Q.transpose(1, 2) # [4, 4, 16, 16]
K = K.transpose(1, 2) # [4, 4, 16, 16]
V = V.transpose(1, 2) # [4, 4, 16, 16]

In [13]:
# Calculate QK^T Attention
# Calculate the attention score betwee Q and K^T
attention_score = torch.matmul(Q, K.transpose(-2, -1))

In [26]:
# Then Scale the attention score by the square root of the head size
attention_score = attention_score / torch.sqrt(torch.tensor(d_model // num_heads, dtype=torch.float))
# pd.DataFrame(attention_score[0][0].detach().cpu().numpy())

In [27]:
# Apply Mask to attention scores
attention_score = attention_score.masked_fill(torch.triu(torch.ones(attention_score.shape[-2:]), diagonal=1).bool(), float('-inf')) #[4, 4, 16, 16] [batch_size, num_heads, context_length, context_length]
# pd.DataFrame(attention_score[0][0].detach().cpu().numpy())

In [19]:
# Softmax the attention score
attention_score = torch.softmax(attention_score, dim=-1)  #[4, 4, 16, 16] [batch_size, num_heads, context_length, context_length]

In [20]:
# Calculate the V attention output
A = torch.matmul(attention_score, V) # [4, 4, 16, 16] [batch_size, num_heads, context_length, head_size]
A.shape

torch.Size([4, 4, 16, 16])

In [21]:
A = A.transpose(1, 2) # [4, 16, 4, 16] [batch_size, context_length, num_heads, head_size]
A = A.reshape(batch_size, -1, d_model) # [4, 16, 64] [batch_size, context_length, d_model]

In [22]:
# Define the output weight matrix
Wo = nn.Linear(d_model, d_model)
output = Wo(A) # [4, 16, 64] [batch_size, context_length, d_model]

print(output.shape)

torch.Size([4, 16, 64])


In [23]:
# Add residual connection
output = output + X

# Add Layer Normalization
layer_norm = nn.LayerNorm(d_model)
output = layer_norm(output)

In [24]:
# Define Feed Forward Network
output = nn.Linear(d_model, d_model * 4)(output)
output = nn.ReLU()(output)
output = nn.Linear(d_model * 4, d_model)(output)
output = torch.dropout(output, p=dropout, train=True)

In [25]:
# Add residual connection
output = output + X
# Add Layer Normalization
layer_norm = nn.LayerNorm(d_model)
output = layer_norm(output)

In [63]:
# apply final linear layer to get the logits
logits = nn.Linear(d_model, max_token_value+1)(output)
logits.shape

torch.Size([4, 16, 100070])

In [65]:
possibilities = F.softmax(logits, dim=-1)
predicted_index = torch.argmax(possibilities[0, 0]).item()
print(predicted_index)
encoding.decode([predicted_index])

68273


'tog'